In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from tensorflow.keras.models import load_model
# Load the trained ANN model
ann_model = load_model('/content/drive/MyDrive/MasterThesis/model_ann_flickr.h5')
ann_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               256256    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 297473 (1.13 MB)
Trainable params: 297473 (1.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import json

# Define the path to your JSON file
file_path = '/content/drive/My Drive/MasterThesis/flickr8k_dataset/combined_captions_data_flickr_final.json'

# Open and load the JSON data
with open(file_path, 'r') as file:
    captions_data = json.load(file)


In [ ]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Resize, Normalize, ToTensor
from pathlib import Path
import random
from collections import defaultdict
import matplotlib.pyplot as plt
from torchvision.transforms.functional import to_pil_image
import os
import torchvision.transforms as transforms

class Flickr8kDataset(Dataset):
    def __init__(self, captions_file, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.img_captions = defaultdict(list)  # Use defaultdict to automatically handle lists of captions

        # Read the .txt file
        with open(captions_file, 'r') as file:
            for line in file:
                parts = line.strip().split(',', 1)  # Split on the first comma
                if len(parts) == 2:
                    img_name, caption = parts
                    self.img_captions[img_name].append(caption)

        # Convert img_captions keys to a list to index images
        self.img_names = list(self.img_captions.keys())
        print(self.img_captions.items())

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_name = self.img_names[idx]
        captions = self.img_captions[img_name]  # Get all captions for the image
        print(captions)
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, captions

# Transformation pipeline with data augmentation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Initialize your dataset and DataLoader
dataset = Flickr8kDataset('/content/drive/My Drive/MasterThesis/flickr8k_dataset/captions.txt', '/content/drive/My Drive/MasterThesis/flickr8k_dataset/Images', transform)
data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import torch
import numpy as np
import requests
from PIL import Image
from io import BytesIO
import torchvision.transforms as transforms
from torchvision.models import resnet50
from transformers import CLIPProcessor, CLIPModel, pipeline
import nltk
from tensorflow.keras.models import load_model
from nltk.tokenize import sent_tokenize, word_tokenize
import json
from tqdm import tqdm

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Initialize the summarization pipeline and CLIP components
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

# Load the trained ANN model
ann_model = load_model('/content/drive/MyDrive/MasterThesis/model_ann_flickr.h5')

# Pre-trained ResNet50 model for feature extraction
resnet_model = resnet50(pretrained=True)
resnet_model.eval()

# Transformation pipeline with data augmentation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def download_image(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content))
            return img
        else:
            return None
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return None

def extract_features(image, model):
    with torch.no_grad():
        features = model(image)
    return features.numpy().flatten()

# Function to select key phrases from text
def select_key_phrases(text, num_words=3):
    words = word_tokenize(text)
    # Select the first few words based on num_words
    selected_words = ' '.join(words[:num_words])
    return selected_words

# Function to create combined text for summarization
def create_combined_text_for_summarization(gpt2_caption, blip_caption, gpt2_weight, blip_weight):
    if gpt2_weight > blip_weight:
        primary_caption = gpt2_caption
        secondary_caption = select_key_phrases(blip_caption, 5)  # Take only a few words from the less weighted caption
    else:
        primary_caption = blip_caption
        secondary_caption = select_key_phrases(gpt2_caption, 5)  # Take only a few words from the less weighted caption

    combined_text = primary_caption + ' ' + secondary_caption
    return combined_text


# Incremental image ID
image_id = 0

# Assuming `captions_data` is a loaded JSON containing your dataset
for images, _ in tqdm(data_loader):

        features = extract_features(images, resnet_model).reshape(1, -1)
        predicted_prob = ann_model.predict(features)
        predicted_label = (predicted_prob > 0.5).astype(int)

        print(predicted_label)
        # Determine weights based on prediction
        gpt2_weight, blip_weight = (2, 1) if predicted_label == 0 else (1, 2)

        item = captions_data[str(image_id)]

        gpt2_caption = item['generated_captions'].get('gpt2', '')
        blip_caption = item['generated_captions'].get('blip', '')

        combined_text = create_combined_text_for_summarization(gpt2_caption, blip_caption, gpt2_weight, blip_weight)

        input_length = len(combined_text.split())
        max_length_ratio = 0.75
        min_viable_length = 20
        max_length = max(min_viable_length, int(input_length * max_length_ratio))
        min_length = max(10, max_length // 2)

        summarized_caption = summarizer(combined_text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']

        # Encode the summarized caption and original captions to get embeddings
        inputs = clip_processor(text=[summarized_caption] + item['original_coco_captions'], return_tensors="pt", truncation=True, padding=True)
        embeddings = clip_model.get_text_features(**inputs)

        # Calculate cosine similarity between the summarized caption and original captions
        similarities = [torch.cosine_similarity(embeddings[0], emb, dim=0).cpu().item() for emb in embeddings[1:]]
        most_similar_index = similarities.index(max(similarities))
        most_similar_caption = item['original_coco_captions'][most_similar_index]
        cosine_similarity_score = max(similarities)

        # Update the JSON structure with new information
        item['summarized_caption'] = summarized_caption
        item['most_similar_caption'] = most_similar_caption
        item['cosine_similarity_score'] = cosine_similarity_score

        print(f"Summarized Caption: {summarized_caption}")
        print(f"Most Similar Original Caption: {most_similar_caption}")
        print(f"Cosine Similarity Score: {cosine_similarity_score}")
        print("=" * 50)

        # Increment the image ID for the next iteration
        image_id += 1
        if image_id==1100:
          break

# Specify the output path for the updated JSON data
output_path = '/content/drive/MyDrive/MasterThesis/weighted_summarization_ann_flickr_modified.json'  # Update this path as needed

# Save the modified JSON data to a new file
with open(output_path, 'w') as file:
    json.dump(captions_data, file, indent=4)

print("JSON file has been updated and saved.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
  0%|          | 0/8091 [00:00<?, ?it/s]

['A child in a pink dress is climbing up a set of stairs in an entry way .', 'A girl going into a wooden building .', 'A little girl climbing into a wooden playhouse .', 'A little girl climbing the stairs to her playhouse .', 'A little girl in a pink dress going into a wooden cabin .']
1/1 [==============================] - 0s 73ms/step


Your max_length is set to 20, but your input_length is only 17. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)


[[0]]


  0%|          | 1/8091 [00:03<7:27:08,  3.32s/it]

...

Summarized Caption:  a boy is sitting on a boat in the water  there is a young boy .
Most Similar Original Caption: "A boy wearing a black and blue life vest is on a green , wooden boat looking out towards the water ."
Cosine Similarity Score: 0.804581344127655
['A child in a white shirt standing in a field of grass .', 'A small boy is playing in an open field .', 'A young boy in a white dress shirt is playing on a grassy plain .', 'A young boy in a white shirt in a field', 'A young boy wearing a t-shirt and jeans is walking in a grassy field .']
1/1 [==============================] - 0s 23ms/step
[[1]]


 14%|█▎        | 1099/8091 [56:04<5:42:50,  2.94s/it]

Summarized Caption:  There is a young boy that is playing with a frisbee in a field a
Most Similar Original Caption: A small boy is playing in an open field .
Cosine Similarity Score: 0.8038101196289062
['A fluffy white dog and two white sheep smell each other .', 'A small white dog inspects two sheep .', 'A white dog is sniffing a sheep through a fence .', 'A white dog sniffing sheep through a fence .', 'White fluffy dog kissing sheep through a fence .']
1/1 [==============================] - 0s 22ms/step
[[1]]


 14%|█▎        | 1099/8091 [56:07<5:57:04,  3.06s/it]

Summarized Caption:  There is a white dog standing next to a sheep in a fenced in area a
Most Similar Original Caption: A white dog is sniffing a sheep through a fence .
Cosine Similarity Score: 0.8698167204856873
JSON file has been updated and saved.
